In [1]:
# Imports
import time
import numpy as np
from six import next
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import scipy
import pickle
#import _pickle as cPickle
import codecs

In [2]:
df = pd.read_csv(r"/scratch/user/achadha7/IRProject/MovieLens/ml-latest-small/ratings.csv", sep=',', engine='python')

In [3]:
df["rating"] = df["rating"].astype(np.float32)

In [4]:
user_mapping = {}
movie_mapping = {}
index = 0
for x in list(df["userId"].unique()):
    user_mapping[x] = index
    index += 1
index = 0
for x in list(df["movieId"].unique()):
    movie_mapping[x] = index
    index += 1

In [5]:
df["userId"] = df["userId"].map(user_mapping)
df["movieId"] = df["movieId"].map(movie_mapping)

In [6]:
df.shape

(100004, 4)

In [8]:
df.head()

,userId,movieId,rating,timestamp
0,0,0,2.5,1260759144
1,0,1,3.0,1260759179
2,0,2,3.0,1260759182
3,0,3,2.0,1260759185
4,0,4,4.0,1260759205


In [9]:
movies = pd.read_csv(r"/scratch/user/achadha7/IRProject/MovieLens/ml-latest-small/movies.csv", sep=',', engine='python')
movies["movieId"]= movies["movieId"].map(movie_mapping)
movies = movies.set_index('movieId')
movies["genres"]= movies["genres"].map(lambda x: x.replace('|', ' ').lower())

In [12]:
movies.head(10)

,title,genres
movieId,,
417.0,Toy Story (1995),adventure animation children comedy fantasy
650.0,Jumanji (1995),adventure children fantasy
319.0,Grumpier Old Men (1995),comedy romance
2084.0,Waiting to Exhale (1995),comedy drama romance
651.0,Father of the Bride Part II (1995),comedy
652.0,Heat (1995),action crime thriller
2061.0,Sabrina (1995),comedy romance
2518.0,Tom and Huck (1995),adventure children
2062.0,Sudden Death (1995),action


In [13]:
movie_content = []
index_set = set(movies.index)
for i in range(len(movie_mapping)):
    if i in index_set:
        movie_content.append(movies.loc[[i]].iloc[0]["genres"])
    else:
        movie_content.append('')

In [14]:
movie_content

['drama',
 'animation children drama musical',
 'thriller',
 'action adventure sci-fi thriller',
 'drama',
 'drama war',
 'action adventure drama',
 'drama',
 'fantasy horror romance thriller',
 'thriller',
 'adventure sci-fi',
 'adventure animation comedy crime',
 'action crime thriller',
 'action adventure sci-fi',
 'adventure comedy',
 'action adventure fantasy',
 'adventure animation children comedy fantasy',
 'drama horror sci-fi thriller',
 'adventure comedy fantasy sci-fi',
 'comedy western',
 'action adventure thriller',
 'drama romance',
 'comedy romance',
 'mystery thriller',
 'crime mystery thriller',
 'comedy drama romance',
 'drama',
 'action drama war',
 'comedy',
 'adventure drama imax',
 'action adventure comedy crime',
 'drama thriller war',
 'action crime thriller',
 'action drama romance',
 'action crime thriller',
 'comedy romance',
 'action adventure sci-fi',
 'drama romance',
 'comedy',
 'drama thriller',
 'comedy drama',
 'comedy',
 'drama horror',
 'drama',
 'dr

In [15]:
index_set

{0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0

In [16]:
vectorizer = CountVectorizer(binary = True)
movie_content = vectorizer.fit_transform(movie_content)
movie_content = movie_content.astype(np.float32)

In [17]:
movie_content

<9066x24 sparse matrix of type '<class 'numpy.float32'>'
	with 21178 stored elements in Compressed Sparse Row format>

In [18]:
users = pd.read_csv(r"/scratch/user/achadha7/IRProject/MovieLens/ml-latest-small/tags.csv", sep=',', engine='python')
users["userId"]= users["userId"].map(user_mapping)
users = users.set_index('userId')
users.head()

,movieId,tag,timestamp
userId,,,
14,339,sandra 'boring' bullock,1138537770
14,1955,dentist,1193435061
14,7478,Cambodia,1170560997
14,32892,Russian,1170626366
14,34162,forgettable,1141391765


In [19]:
user_content = []
index_set = set(users.index)
for i in range(len(user_mapping)):
    if i in index_set:
        user_content.append(' '.join(list(users.loc[[i]]["tag"])))
    else:
        user_content.append('')
user_content = vectorizer.fit_transform(user_content)
user_content = user_content.astype(np.float32)

In [20]:
user_content

<671x764 sparse matrix of type '<class 'numpy.float32'>'
	with 1137 stored elements in Compressed Sparse Row format>

In [53]:
df.head()

,userId,movieId,rating,timestamp
0,0,0,2.5,1260759144
1,0,1,3.0,1260759179
2,0,2,3.0,1260759182
3,0,3,2.0,1260759185
4,0,4,4.0,1260759205


In [57]:
m = df.shape[0]

In [62]:
half_size = int(m/2)
print(half_size)

50002


In [64]:
df1 = df.iloc[:half_size]
df2 = df.iloc[half_size:]

In [67]:
df1.head()

,userId,movieId,rating,timestamp
0,0,0,2.5,1260759144
1,0,1,3.0,1260759179
2,0,2,3.0,1260759182
3,0,3,2.0,1260759185
4,0,4,4.0,1260759205


In [74]:
df1.isnull().values.any()
df2.isnull().values.any()

False

In [75]:
index1 = df1.index
df2.set_index(index1)

,userId,movieId,rating,timestamp
0,366,735,3.5,1128633117
1,366,2166,5.0,1128632828
2,366,4789,3.0,967059919
3,366,170,4.5,1128632716
4,366,173,4.0,1128632823
5,366,176,3.0,997908852
6,366,179,5.0,1128632771
7,366,105,2.0,1128632782
8,366,180,5.0,1128632801
9,366,181,5.0,1128632705


Before: (50002, 6)
After: (50002, 6)


In [ ]:
df_train = df_n[0:split_index_train]
df_val = df_n[split_index_train : split_index_val]
df_test = df_n[split_index_val:].reset_index(drop=True)